In [2]:
import pandas as pd

# Realizando a leitura do arquivo de avaliações dos filmes a serem analisados
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('Aulas_Codigo_e_Exercicios/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))

# Exibindo as primeiras linhas do arquivo de avaliações em um DataFrame
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [3]:
import numpy as np

# Agrupando as avaliações por filme e calculando a média e a quantidade de avaliações
movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()


C:\Users\cainh\AppData\Local\Temp\ipykernel_92180\1930408656.py:4: FutureWarning: The provided callable <function mean at 0x000001FCFF505F80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [4]:
# Normalizando a quantidade de avaliações por filme, utilizando a fórmula de normalização min-max
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])

# Aplicando a normalização min-max
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

# Exibindo as primeiras linhas do DataFrame com a quantidade de avaliações normalizadas
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [6]:
movieDict = {}

# abrindo o arquivo referente aos filmes e armazenando as informações em um dicionário para posterior manipulação

with open(r'Aulas_Codigo_e_Exercicios/ml-100k/u.item', encoding="ISO-8859-1") as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))

In [7]:
from scipy import spatial

# Função para calcular a "distância" entre dois filmes
def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB) # calculando a distância entre os gêneros dos filmes utilizando a distância cosseno
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB) # calculando a distância entre a popularidade dos filmes
    return genreDistance + popularityDistance
    
# Calculando a distância entre os filmes com ID 2 e 4
ComputeDistance(movieDict[2], movieDict[4])

0.8004574042309892

In [9]:
import operator

# Função para obter os K vizinhos mais próximos de um filme

def getNeighbors(movieID, K):
    distances = []
    
    # Percorrendo todos os filmes
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie]) # definindo a distância entre os filmes
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1)) # ordenando as distâncias
    neighbors = [] # armazenando os K vizinhos mais próximos

    # Adicionando os K vizinhos mais próximos em neighbors
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10 # definindo o número de vizinhos mais próximos, o k.
avgRating = 0 # inicializando a variável para armazenar a média das avaliações dos vizinhos
neighbors = getNeighbors(1, K) # utilizando a função getNeighbors para obter os K vizinhos mais próximos do filme 1

# Exibindo os K vizinhos mais próximos do filme 1 e a média das avaliações desses vizinhos
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


In [10]:
# Função para definir a distância entre dois filmes, utilizando a distância de correlação
def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.correlation(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance

In [11]:
K = 12
avgRating = 0
neighbors = getNeighbors(1, K)
print(movieDict[1][1])
# Imprime os 12 filmes mais próximos ao filme 1
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]) + "    " + str(movieDict[neighbor][1]))
    
avgRating /= K
    

movieDict[1]


[0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Liar Liar (1997) 3.156701030927835    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Aladdin (1992) 3.8127853881278537    [0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0]
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975    [0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Monty Python and the Holy Grail (1974) 4.0664556962025316    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Full Monty, The (1997) 3.926984126984127    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
George of the Jungle (1997) 2.685185185185185    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Beavis and Butt-head Do America (1996) 2.7884615384615383    [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Birdcage, The (1996) 3.4436860068259385    [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Aladdin and the King of Thieves (1996) 2.8461538461538463    [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Home Alone (1990) 3.0875912408759123    [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Jungle2Jungle (1997) 2.4393939393939394    [0 0 0 0 1 1 0 0

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)